In [1]:
! pip install influxdb-client

     ---------------------------------------- 0.0/64.8 kB ? eta -:--:--
     ------ --------------------------------- 10.2/64.8 kB ? eta -:--:--
     ------ --------------------------------- 10.2/64.8 kB ? eta -:--:--
     ------------------------ ------------- 41.0/64.8 kB 393.8 kB/s eta 0:00:01
     -------------------------------------- 64.8/64.8 kB 496.1 kB/s eta 0:00:00
   ---------------------------------------- 0.0/745.4 kB ? eta -:--:--
   --- ------------------------------------ 61.4/745.4 kB 1.7 MB/s eta 0:00:01
   ------- -------------------------------- 143.4/745.4 kB 1.4 MB/s eta 0:00:01
   ---------- ----------------------------- 194.6/745.4 kB 1.7 MB/s eta 0:00:01
   ---------------------- ----------------- 419.8/745.4 kB 2.4 MB/s eta 0:00:01
   -------------------------- ------------- 501.8/745.4 kB 2.2 MB/s eta 0:00:01
   ---------------------------------------- 745.4/745.4 kB 2.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/217.8 kB ? eta -:--:--
 


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
import influxdb_client
from influxdb_client.client.write_api import SYNCHRONOUS
from datetime import datetime
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

# Configuration
url = os.getenv("INFLUXDB_URL")
token = os.getenv("INFLUXDB_TOKEN")
org = os.getenv("INFLUXDB_ORG")
bucket = os.getenv("INFLUXDB_BUCKET")

# Initialize the InfluxDB client
client = influxdb_client.InfluxDBClient(url=url, token=token, org=org)
write_api = client.write_api(write_options=SYNCHRONOUS)

# Helper function to create a data point
def create_data_point(player_id, game_id, metric_type, value, timestamp=None):
    timestamp = timestamp or datetime.utcnow()
    point = influxdb_client.Point("game_metrics") \
        .tag("player_id", player_id) \
        .tag("game_id", game_id) \
        .field(metric_type, value) \
        .time(timestamp)
    return point

def insert_metric(player_id, game_id, metric_type, value):
    point = create_data_point(player_id, game_id, metric_type, value)
    write_api.write(bucket=bucket, org=org, record=point)
    print(f"Inserted {metric_type} data for player {player_id} in game {game_id}")

# Example Usage
insert_metric("player1", "game1", "score", 500)
insert_metric("player2", "game2", "score", 400)
insert_metric("player3", "game1", "score", 800)

C:\Users\ANITA\AppData\Local\Temp\ipykernel_18572\2732666945.py:17: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = timestamp or datetime.utcnow()


Inserted score data for player player1 in game game1
Inserted score data for player player2 in game game2
Inserted score data for player player3 in game game1


In [15]:
import influxdb_client

# Configuration (same as above)
query_api = client.query_api()

def get_metrics(player_id, game_id, start_time, end_time):
    query = f'''
    from(bucket: "{bucket}")
        |> range(start: {start_time}, stop: {end_time})
        |> filter(fn: (r) => r["player_id"] == "{player_id}")
        |> filter(fn: (r) => r["game_id"] == "{game_id}")
    '''
    result = query_api.query(org=org, query=query)
    metrics = []
    for table in result:
        for record in table.records:
            metrics.append({
                "time": record.get_time(),
                "metric": record.get_field(),
                "value": record.get_value()
            })
    return metrics

# Example Usage
start = "2024-08-28T00:00:00Z"
end = "2024-08-30T00:00:00Z"
metrics = get_metrics("player1", "game1", start, end)
print("Retrieved Metrics:", metrics)


Retrieved Metrics: [{'time': datetime.datetime(2024, 8, 29, 15, 15, 28, 976522, tzinfo=datetime.timezone.utc), 'metric': 'score', 'value': 500}, {'time': datetime.datetime(2024, 8, 29, 15, 21, 21, 301484, tzinfo=datetime.timezone.utc), 'metric': 'score', 'value': 500}, {'time': datetime.datetime(2024, 8, 29, 15, 30, 10, 26375, tzinfo=datetime.timezone.utc), 'metric': 'score', 'value': 600}]


In [10]:
from datetime import timedelta

def update_metric(player_id, game_id, metric_type, new_value, timestamp):
    # Deleting the existing point
    # delete_metric(player_id, game_id, metric_type, timestamp)
    
    # Inserting the new point with updated value
    insert_metric(player_id, game_id, metric_type, new_value)

# Example Usage
timestamp_to_update = datetime.utcnow()
update_metric("player1", "game1", "score", 600, timestamp_to_update)


C:\Users\ANITA\AppData\Local\Temp\ipykernel_18572\1883246090.py:11: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp_to_update = datetime.utcnow()
C:\Users\ANITA\AppData\Local\Temp\ipykernel_18572\2732666945.py:17: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = timestamp or datetime.utcnow()


Inserted score data for player player1 in game game1
